In [1]:
from tensorflow.python.keras.layers import Bidirectional, Dropout, Activation, Dense, LSTM,GRU
 
from tensorflow.keras.models import Sequential
from tensorflow.python.keras.callbacks import EarlyStopping
 
from sklearn.preprocessing import MinMaxScaler
 
from datetime import datetime

from binance.client import Client
from binance.enums import *

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import time


In [2]:
client = Client('   ', '  ')

In [3]:
cmc = CoinMarketCapAPI('  ')

In [4]:
klines = client.get_historical_klines('BNBUSDT', Client.KLINE_INTERVAL_15MINUTE, "10 day ago UTC") 

In [5]:
len(klines)

960

In [6]:
info = client.get_all_tickers()

In [7]:
quote_ccy = 'USDT'

In [8]:
ticker_list = [d['symbol'] for d in info if d['symbol'].endswith(quote_ccy)]

In [9]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [10]:
target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 300 and d['rank'] <= 500]

In [11]:
target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [12]:
def is_candidate(filtered_klines_info, tolerable_volume):
    context_klines_info = filtered_klines_info[:len(filtered_klines_info) -1]
    is_candidate = False
    
    trigger_kline = filtered_klines_info[-1]
    
    for context_kline in context_klines_info:
        kontext_tol_val = tolerable_volume * 1.75
        
        volume_gate = float(context_kline[3])  <  kontext_tol_val
        if volume_gate :
            is_candidate = True
        else:
            return False


    trigger_gate = float(trigger_kline[3]  > tolerable_volume * 10) and (trigger_kline[-1] < 9 ) and (trigger_kline[-1] > 5 ) 
    
    

    if trigger_gate:
        is_candidate = True
    else:
        is_candidate = False

    return is_candidate

In [13]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[2] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [14]:
for target_ticker in target_ticker_list:
    
    print('Analyzing Hourly Started ')
    print(target_ticker)

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_15MINUTE, "30 day ago UTC") 
    length = int(len(klines))
    if(length < 96):
        break
    
    array = np.array(klines, dtype=np.float64)
    focused_kline_array = np.delete(array, [2, 3, 6, 7, 8, 9, 10, 11], 1)
    percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
    percentage_value = np.reshape(percentage_value, (length, 1))
    focused_kline_array = np.append(focused_kline_array, percentage_value, 1)
    average_volume = np.mean(focused_kline_array[:,3])
    tolerable_volume = average_volume * 1.2
    

    for x in range(0, length):
        

        filtered_klines_info = focused_kline_array[x:x+96]
        buy_kline_info = filtered_klines_info[-1]
        if(len(filtered_klines_info) == 96 and is_candidate(filtered_klines_info, tolerable_volume)):
            
            print('********** SUCCESS ************************')
            print(target_ticker)
            print('Buy Price')
            print(buy_kline_info[2])
            print('Kline TS')
            print(str(buy_kline_info[0]))
            

            busd = 10
            
            quantity = busd/ buy_kline_info[2]
            print('Quantity')
            print(quantity)
            quantity_str = "{:0.0{}f}".format(quantity, 2)
            print(quantity_str)
            
            """
            order = client.create_test_order(symbol=target_ticker, side=SIDE_BUY, type='MARKET', quoteOrderQty=busd)
            time.sleep(5)
            order_id = order['orderId'] 
            '''order_id = client.get_all_orders(symbol=target_ticker)[0]['orderId'] '''
   
            buy_price = order['fills'][0]['price']
            buy_qty = order['executedQty']
            sell_price = buy_price * 1.03
            
            
            
            while client.get_order(symbol=target_ticker, orderId=order_id)['status'] != 'FILLED':
                time.sleep(10)
                
            if client.get_order(symbol=target_ticker, orderId=order_id)['status'] == 'FILLED':
                asset = target_ticker.split("USDT")[0]
                balance = client.get_asset_balance(asset=asset)
                free_to_trade = balance['free']
                sell_price_str = "{:0.0{}f}".format(sell_price, 2)
                
                order = client.create_test_order(symbol=target_ticker, side=SIDE_SELL, type='LIMIT', quantity=float(balance['free']), price=sell_price_str, timeInForce=TIME_IN_FORCE_GTC)
                print('********** SELL ORDER SET ************************')
                
                """


Analyzing Hourly Started 
FETUSDT
********** SUCCESS ************************
FETUSDT
Buy Price
0.0775
Kline TS
1656750600000.0
Quantity
129.03225806451613
129.03
Analyzing Hourly Started 
DUSKUSDT
Analyzing Hourly Started 
COCOSUSDT
Analyzing Hourly Started 
TOMOUSDT
Analyzing Hourly Started 
MFTUSDT
Analyzing Hourly Started 
WANUSDT
Analyzing Hourly Started 
BANDUSDT
Analyzing Hourly Started 
NKNUSDT
Analyzing Hourly Started 
ARPAUSDT
Analyzing Hourly Started 
TROYUSDT
Analyzing Hourly Started 
BTSUSDT
Analyzing Hourly Started 
LTOUSDT
Analyzing Hourly Started 
MBLUSDT
Analyzing Hourly Started 
VTHOUSDT
Analyzing Hourly Started 
BLZUSDT
Analyzing Hourly Started 
ANTUSDT
Analyzing Hourly Started 
YFIIUSDT
Analyzing Hourly Started 
DIAUSDT
Analyzing Hourly Started 
BELUSDT
Analyzing Hourly Started 
SUNUSDT
Analyzing Hourly Started 
FLMUSDT
********** SUCCESS ************************
FLMUSDT
Buy Price
0.1299
Kline TS
1655722800000.0
Quantity
76.98229407236336
76.98
Analyzing Hourly Star

In [15]:
order = client.create_test_order(symbol='RADUSDT', side=SIDE_BUY, type='MARKET', quoteOrderQty=10)

In [16]:
order

{}

In [17]:
sell_price = float(order['fills'][0]['price']) * 1.03

KeyError: 'fills'

In [ ]:
sell_price_str = "{:0.0{}f}".format(sell_price, 2)

In [ ]:
sell_price_str

In [ ]:
order['status']

In [ ]:
client.get_order(symbol='RADUSDT', orderId='66488231')

In [ ]:
order

In [ ]:
symbol = 'RADUSDT'

In [ ]:
symbol.split("USDT")[0]

In [ ]:
balance = client.get_asset_balance(asset='RAD')

In [ ]:
float(balance['free'])

In [ ]:
order = client.create_order(symbol='RADUSDT', side=SIDE_SELL, type='LIMIT', quantity=float(balance['free']), price=sell_price_str, timeInForce=TIME_IN_FORCE_GTC)

In [ ]:
full_test_data_set = []

for key in hit_ticker_dictionary.keys():
    key_test_data_set = []
    key_test_data_set.append(key) 
    trigger_time_in_ms = hit_ticker_dictionary[key][0][0]
    key_test_data_set.append(trigger_time_in_ms) 
    open_price = hit_ticker_dictionary[key][0][4]
    key_test_data_set.append(open_price)
    analysis_kline_list = client.get_historical_klines(symbol=key, 
                                                       interval=Client.KLINE_INTERVAL_1HOUR, 
                                                       start_str=trigger_time_in_ms)[1:6]
    for sell_kline in analysis_kline_list:
        close_price = sell_kline[4]

        key_test_data_set.append(close_price)
        diff = (float(close_price) - float(open_price))
        profit = diff/float(open_price) * 100
        key_test_data_set.append(profit)
        
    full_test_data_set.append(key_test_data_set)
    

In [ ]:
full_test_data_set